### Packages

In [ ]:
## packages
import os
import numpy as np
import pandas as pd
import re

### Real-Time Data Processing

In [ ]:
# Natural sorting 
def natural_sort_key(s):
    _nsre = re.compile('([0-9]+)')
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(_nsre, s)]  
## set of function to read and process the raw spectra
def read_files(loc):
    files_all = sorted(os.listdir(loc), key = natural_sort_key)
    ABS_names = []
    PL_names = []
    ABS_all = []
    PL_all = []
    for file in files_all:
        if (file.startswith("Abs")):
            ABS = pd.read_csv(loc + "//" + file , header = None).to_numpy()
            ABS_all.append(ABS)
            ABS_names.append(file.split(".")[0])
        elif (file.startswith("PL")):
            PL = pd.read_csv(loc + "//" + file, header = None).to_numpy()
            PL_all.append(PL)
            PL_names.append(file.split(".")[0])
        elif(file.startswith("Wavelength_Abs")):
            WL_ABS = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("Wavelength_PL")):
            WL_PL = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("DR_Abs")):
            DR_ABS = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("DR_PL")):
            DR_PL = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("LR")):
            LR = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("FR")):
            FR = pd.read_csv(loc + "//" + file, header = None).to_numpy()    
    return WL_ABS, WL_PL, DR_ABS, DR_PL, LR, ABS_all, PL_all, ABS_names, PL_names, FR
def idx_min(y, x):
    diff = np.abs(y[:, 0] - x)
    idx = diff.argmin()
    return idx
# extract reactive phase
def extract(x, idx1, idx2, numFirstElements):
    x_mean = x[idx1:idx2 + 1, :].mean(axis = 0)
    x_sort = np.sort(x_mean)
    idx_phase = np.nonzero(np.in1d(x_mean, x_sort[:numFirstElements]))[0]
    x_phase = x[:, idx_phase]
    return x_phase
def spectra_avg(x):
    x_avg = x.mean(axis = 1) # avg over extracted reactive phase spectra for each WL; will result in a row matrix
    x_avg = x_avg[:, np.newaxis]
    return x_avg
def baseline_zero(x, idx_low, idx_high):
    x_baseline_zero = x - x[idx_low:idx_high + 1, :].mean() # make baseline zero; subtracting mean of PL at LL - HL nm
    return x_baseline_zero
def linear_int_x(y, x, i, y_btw):
    x_btw = x[i, 0] - ((x[i + 1, 0] - x[i, 0]) / (y[i + 1, 0] - y[i, 0])) * (y[i, 0] - y_btw)
    return x_btw
def linear_int_y(y, x, i, x_btw):
    y_btw = y[i, 0] - ((y[i + 1, 0] - y[i, 0]) / (x[i + 1, 0] - x[i, 0])) * (x[i, 0] - x_btw)
    return y_btw
def peak_info(y, x, emission_intensity):
    # correction for emission peak intensity in case min is not exactly zero
    # min_intensity = y.min()
    min_intensity = 0 # without correction
    intensity_peak = emission_intensity - min_intensity
    # emission peak area
    area_peak = np.trapz(y.T, x = x.T)[0]
    # emission peak intensity and area
    output = [intensity_peak, area_peak]
    return np.array(output)[:, np.newaxis]
# process ABS
def spectra_extract_ABS(WL, DR, LR, ABS_all, ABS_names):
    # baseline WL limits
    baseline_LL = 700
    baseline_HL = 800
    idx_WL_LL = idx_min(WL, baseline_LL)
    idx_WL_HL = idx_min(WL, baseline_HL)
    # to extract reactive phase
    lastPeak_LL = 250
    lastPeak_HL = 350 
    idx_lastPeak_LL = idx_min(WL, lastPeak_LL)
    idx_lastPeak_HL = idx_min(WL, lastPeak_HL)
    # excitation WL info
    excitation_WL = 300
    idx_excitationWL = idx_min(WL, excitation_WL)
    # initiate files to be exported
    WLABS_processed = WL
    ABS_excitationWL_list = []
    # ABS_excitationWL_list = np.array([["Absorbance at excitation WL"]])
    for (item, name) in zip(ABS_all, ABS_names):
        ABS_only = item[1:, :]
        ## extract reactive phase and remove carrier phase (PFO)
        param = 50
        ABS_phase = extract(ABS_only, idx_lastPeak_LL, idx_lastPeak_HL, param)
        ABS_phase_avg = spectra_avg(ABS_phase) 
        arg_log = (ABS_phase_avg - DR) / (LR - DR) # Beer-Lambert law
        ABS_processed = - np.log10(arg_log) # Beer-Lambert law
        ABS_processed_baseline = baseline_zero(ABS_processed, idx_WL_LL, idx_WL_HL) 
        WLABS_processed = np.concatenate( [WLABS_processed, ABS_processed_baseline] , axis = 1) # attach WL and processed ABS
        # WLABS_processed_filtered = WLABS_processed[np.isnan(WLABS_processed[:, 1]) == False] # remove NaN ABS
        ABS_excitationWL = linear_int_y(ABS_processed_baseline, WL, idx_excitationWL, excitation_WL)
        ABS_excitationWL_list.append(ABS_excitationWL)
        # ABS_excitationWL_list = np.concatenate( [ABS_excitationWL_list, np.array([ABS_excitationWL])[:, np.newaxis]] , axis = 1)
    ABS_excitationWL_list = np.array(ABS_excitationWL_list)[:, np.newaxis]
    return WLABS_processed, ABS_excitationWL_list # processed ABS spectra as well as the Abs300nm as one one of the optical features of interest
    # return ABS_excitationWL_list
# process PL
def spectra_extract_PL(WL, DR, PL_all, PL_names):
    ## baseline WL limits
    baseline_LL = 800
    baseline_HL = 1000
    idx_WL_LL = idx_min(WL, baseline_LL)
    idx_WL_HL = idx_min(WL, baseline_HL)
    # to extract reactive phase
    excWL_LL = 250
    excWL_HL = 350
    idx_excWL_LL = idx_min(WL, excWL_LL)
    idx_excWL_HL = idx_min(WL, excWL_HL)
    ## emission peak WL and the WL range for the right side of emission peak
    emPeak_WL = 446
    emPeak_LL = 345
    emPeak_HL = 700
    idx_emPeak_LL = idx_min(WL, emPeak_LL)
    idx_emPeak_HL = idx_min(WL, emPeak_HL)
    WL_aroundPeak = WL[idx_emPeak_LL: idx_emPeak_HL + 1, :]
    idx_intensityWL = idx_min(WL, emPeak_WL) # index for emission peak WL
    ## initiate files to be exported
    WLPL_processed = WL
    emission_details_list = []
    # emission_details = np.array([["Emission Peak WL"], ["Emission Peak Intensity"], ["FWHM"], ["Emission Peak Area"]])
    for (item, name) in zip(PL_all, PL_names):
        PL_only = item[1:, :]
        ## extract reactive phase and remove carrier phase (PFO)
        param = 50
        PL_phase = extract(PL_only, idx_excWL_LL, idx_excWL_HL, param) 
        PL_phase_avg = spectra_avg(PL_phase)
        PL_processed = PL_phase_avg - DR
        PL_processed_baseline = baseline_zero(PL_processed, idx_WL_LL, idx_WL_HL) 
        WLPL_processed = np.concatenate( [WLPL_processed, PL_processed_baseline] , axis = 1) # attach WL and processed PL
        ## emission peak info
        PL_emissionWL = linear_int_y(PL_processed_baseline, WL, idx_intensityWL, emPeak_WL)
        PL_aroundPeak = PL_processed_baseline[idx_emPeak_LL: idx_emPeak_HL + 1, :]
        info_peak = peak_info(PL_aroundPeak, WL_aroundPeak, PL_emissionWL)
        emission_details_list.append(info_peak)
        # emission_details = np.concatenate( [emission_details, info_peak_method] , axis = 1)
    emission_details_list = np.concatenate(emission_details_list, axis = 1).T
    return WLPL_processed, emission_details_list # processed PL spectra as well as PLI and PLA as two of the optical features of interest
    # return emission_details_list # emission peak intensity and area as optical features of interest
    # return emission_details_list[:, [1]] # emission peak area as one of the output parameters of interest to calculate relative PLQY
## calculate relative PLQY
def rel_PLQY(y_abs, y_area):
    # Convert y_abs and y_area to pytorch tensors if they are numpy arrays
    if not torch.is_tensor(y_abs):
        y_abs = torch.from_numpy(y_abs.astype(np.float32))
    if not torch.is_tensor(y_area):
        y_area = torch.from_numpy(y_area.astype(np.float32))
    # data related to the dye and solvent
    PLQY_ref = 0.1336
    ABS_ref = 0.9115
    area_ref = 198493.1521
    # calculate relative_PLQY
    PLQY_sample = PLQY_ref * (y_area / area_ref) * ((1 - torch.pow(torch.tensor(10), -ABS_ref)) / (1 - torch.pow(torch.tensor(10), -y_abs)))
    return PLQY_sample

### Master Code

In [ ]:
def master_code():
    path = input("Please enter the path for the directory that includes files: ")
    WL_ABS, WL_PL, DR_ABS, DR_PL, LR, ABS_all, PL_all, ABS_names, PL_names, FR = read_files(path)
    ## process and export the processed absorbance and the corresponding details; absorbance at excitation wavelength
    WLABS_processed, ABS_excitationWL_list = spectra_extract_ABS(WL_ABS, DR_ABS, LR, ABS_all, ABS_names)
    headers_ABS = ["WL"] + ABS_names
    ABS_df = pd.DataFrame(WLABS_processed, columns = headers_ABS)
    ABS_df.to_csv(path + "//" + "ABS_processed.csv", index = False)
    ABS_info_df = pd.DataFrame(ABS_excitationWL_list, columns = ["Absorbance at Excitation Wavelength"], index = ABS_names)
    ABS_info_df.to_csv(path + "//" + "ABS_info.csv")
    ## process and export the processed PL and the corresponding emission details; emission peak intensity and area
    WLPL_processed, emission_details_list = spectra_extract_PL(WL_PL, DR_PL, PL_all, PL_names)
    headers_PL = ["WL"] + PL_names
    PL_df = pd.DataFrame(WLPL_processed, columns = headers_PL)
    PL_df.to_csv(path + "//" + "PL_processed.csv", index = False)
    PL_info_df = pd.DataFrame(emission_details_list, columns = ["Emission Peak Intensity", "Emission Peak Area"], index = PL_names)
    PL_info_df.to_csv(path + "//" + "PL_info.csv")

In [ ]:
master_code()